## Импортируем библиотеки

In [1]:
import pandas as pd
from sqlalchemy import create_engine

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import seaborn as sns
import math
import numpy as np
from scipy import stats as st

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from warnings import simplefilter



## Создаём коннекцию к базе

In [2]:


db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

In [3]:
# получаем сырые данные
query = '''
            SELECT * 
            FROM log_raw
        '''
log_raw = pd.io.sql.read_sql(query, con=engine, index_col='event_id')

log_raw.to_csv('log_raw.csv', index=False)

log_raw.head(5) 

,age_segment,event,item_id,item_topic,item_type,source_id,source_topic,source_type,ts,user_id
event_id,,,,,,,,,,
2931262,18-25,show,1793260,Красота,native,8388799,История,publisher,1569349657638,2185026
2931277,18-25,show,7465641,Культура,native,5837959,История,publisher,1569349657971,6845238
2931285,18-25,show,406073,История,url,2335872,Сад и дача,domain,1569349658408,5244363
2931247,26-30,show,752665,Путешествия,native,6963489,Здоровье,publisher,1569349657189,7553074
2931278,26-30,click,6843081,Юмор,url,5427748,Семейные отношения,domain,1569349658054,1317585


## Изучаем данные

In [4]:
list_df = (log_raw,)
for list_df in list_df:
    display('______________info_____________________')
    display(list_df.info())
    display('______________describe_________________')
    display(list_df.describe())
    display('______________head_____________________')
    display(list_df.head(10))
    display('______________sample___________________')
    display(list_df.sample(20))
    display('______________tail_____________________')
    display(list_df.tail(10))
    display('_____________isna______________________')
    display(list_df.isna().sum().reset_index())
    display('_____________duplicated________________')
    display(list_df.duplicated().sum())
    display('__количество пропущенных значений по каждому из столбцов__')
    display(list_df.isnull().mean().reset_index())

'______________info_____________________'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322391 entries, 2931262 to 4396838
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   age_segment   322391 non-null  object
 1   event         322391 non-null  object
 2   item_id       322391 non-null  int64 
 3   item_topic    322391 non-null  object
 4   item_type     322391 non-null  object
 5   source_id     322391 non-null  int64 
 6   source_topic  322391 non-null  object
 7   source_type   322391 non-null  object
 8   ts            322391 non-null  int64 
 9   user_id       322391 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 27.1+ MB


None

'______________describe_________________'

,item_id,source_id,ts,user_id
count,3.223910e+05,3.223910e+05,3.223910e+05,3.223910e+05
mean,4.907860e+06,4.974883e+06,1.569351e+12,4.988528e+06
std,2.918019e+06,2.865432e+06,3.253844e+05,2.898068e+06
min,1.160000e+02,3.484000e+03,1.569350e+12,1.600000e+01
25%,2.306626e+06,2.613120e+06,1.569351e+12,2.481560e+06
50%,4.874690e+06,4.862937e+06,1.569351e+12,4.957050e+06
75%,7.508017e+06,7.573313e+06,1.569352e+12,7.512082e+06
max,9.999779e+06,9.999497e+06,1.569352e+12,9.999930e+06


...

,index,0
0,age_segment,0
1,event,0
2,item_id,0
3,item_topic,0
4,item_type,0
5,source_id,0
6,source_topic,0
7,source_type,0
8,ts,0
9,user_id,0


'_____________duplicated________________'

0

'__количество пропущенных значений по каждому из столбцов__'

,index,0
0,age_segment,0.0
1,event,0.0
2,item_id,0.0
3,item_topic,0.0
4,item_type,0.0
5,source_id,0.0
6,source_topic,0.0
7,source_type,0.0
8,ts,0.0
9,user_id,0.0


Вывод:
* Данные из БД успешно выгружены в "csv" файл
* Данные содержат 322391  строк и 10 колонок
* Пропусков и дублей нет
* Необходимо преобразовать данные из колонки `ts`


## Подготовка данных

In [5]:
log_raw['event_time'] = pd.to_datetime(log_raw['ts'], unit = 'ms')
log_raw.head()

,age_segment,event,item_id,item_topic,item_type,source_id,source_topic,source_type,ts,user_id,event_time
event_id,,,,,,,,,,,
2931262,18-25,show,1793260,Красота,native,8388799,История,publisher,1569349657638,2185026,2019-09-24 18:27:37.638
2931277,18-25,show,7465641,Культура,native,5837959,История,publisher,1569349657971,6845238,2019-09-24 18:27:37.971
2931285,18-25,show,406073,История,url,2335872,Сад и дача,domain,1569349658408,5244363,2019-09-24 18:27:38.408
2931247,26-30,show,752665,Путешествия,native,6963489,Здоровье,publisher,1569349657189,7553074,2019-09-24 18:27:37.189
2931278,26-30,click,6843081,Юмор,url,5427748,Семейные отношения,domain,1569349658054,1317585,2019-09-24 18:27:38.054


## Изучим данные

In [6]:
dash_visits = log_raw.groupby(['item_topic', 'source_topic','age_segment','event_time'])\
                                                .agg({'user_id': 'count'}).reset_index()
dash_visits.head()

,item_topic,source_topic,age_segment,event_time,user_id
0,Деньги,Авто,18-25,2019-09-24 18:31:40.031,1
1,Деньги,Авто,18-25,2019-09-24 18:31:56.354,1
2,Деньги,Авто,18-25,2019-09-24 18:32:20.475,1
3,Деньги,Авто,18-25,2019-09-24 18:35:02.718,1
4,Деньги,Авто,18-25,2019-09-24 18:53:58.172,1


In [7]:
log_raw.item_type.unique()

array(['native', 'url', 'narrative_card', 'post'], dtype=object)

In [8]:
log_raw.source_type.unique()

array(['publisher', 'domain'], dtype=object)

Сгруппируем темы источника по способу взаимодействия пользователей с системой:

In [9]:
log_raw_source = log_raw.pivot_table(index='source_topic', columns='event', values='item_topic', aggfunc='count')\
                                                            .reset_index().sort_values(by='show', ascending=False)
log_raw_source

event,source_topic,click,show,view
20,Семейные отношения,2879,29678,2057
17,Россия,2251,27161,1608
12,Полезные советы,1957,24424,2156
15,Путешествия,1289,22417,1368
5,Знаменитости,2043,21211,1639
9,Кино,1579,17503,1802
2,Дети,1430,13241,1176
8,История,1101,12885,1190
21,Семья,1207,12072,1186
4,Здоровье,1071,11363,922


In [10]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=log_raw_source['click'],
    x=log_raw_source['source_topic'],
    name='Карточка отображена для пользователя',
    marker_color='#f0932b'
))
fig.add_trace(go.Bar(
    y=log_raw_source['show'],
    x=log_raw_source['source_topic'],
    name='Пользователь кликнул на карточку',
    marker_color='#686de0'
))

fig.add_trace(go.Bar(
    y=log_raw_source['view'],
    x=log_raw_source['source_topic'],
    name='Пользователь просмотрел статью карточки',
    marker_color='#22a6b3'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
        title={
        'text': 'Способы взаимодействия пользователей с системой',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

Сгруппируем статьи по способу взаимодействия пользователей с системой:

In [11]:
log_raw_item = log_raw.pivot_table(index='item_topic', columns='event', values='source_topic', aggfunc='count')\
                                                            .reset_index().sort_values(by='show', ascending=False)
log_raw_item

event,item_topic,click,show,view
11,Наука,1583,19729,1257
13,Отношения,1432,18760,1276
6,Интересные факты,1492,17797,1435
12,Общество,1585,17392,1441
14,Подборки,1241,16002,1200
19,Россия,1260,15401,1016
15,Полезные советы,1149,13776,1132
8,История,1162,13652,1177
20,Семья,850,10787,762
3,Женщины,949,10108,884


In [12]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=log_raw_item['click'],
    x=log_raw_item['item_topic'],
    name='Карточка отображена для пользователя',
    marker_color='#3c40c6'
))
fig.add_trace(go.Bar(
    y=log_raw_item['show'],
    x=log_raw_item['item_topic'],
    name='Пользователь кликнул на карточку',
    marker_color='#0fbcf9'
))

fig.add_trace(go.Bar(
    y=log_raw_item['view'],
    x=log_raw_item['item_topic'],
    name='Пользователь просмотрел статью карточки',
    marker_color='#ff5e57'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
        title={
        'text': 'Способы взаимодействия пользователей с системой',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

Сгруппируем способы взаимодействия пользователей с системой для определения % соотношения:

In [23]:
events_rating = log_raw.groupby('event').agg({'user_id': 'count'})

events_rating['events'] = (log_raw.groupby("event").agg({'user_id':"count"})
         .apply(lambda x: 100*x/x.sum())).round(2)

events_rating.columns = ['количество', '% от общего кол-ва']
events_rating.reset_index().sort_values('% от общего кол-ва', ascending=False)

,event,количество,% от общего кол-ва
1,show,277969,86.22
0,click,23235,7.21
2,view,21187,6.57


Показатель кликабельности

In [21]:
CTR = 23235/277969 * 100
CTR

8.358845770571538

In [22]:
fig = go.Figure(data=go.Pie(labels=events_rating['количество'].value_counts()
                            .reset_index()['index'], values=log_raw['event'].value_counts()))
fig.update_layout(
    title={
        'text':'Соотношение способов взимодействия',
        'y':0.87,
        'x':0.5})
for trace in fig.data:
    trace['labels']=['click', 'show', 'view']
fig.show()

C:\Users\skd23\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\numeric.py:2446: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison




[ссылка на Tableau Public](https://public.tableau.com/app/profile/nikita.semenov/viz/Yandex_Zendashboard_16417409556140/dashboard_zen?publish=yes/)

